# Predicting water baseflow

## Load and clean the database

In [48]:
# imports
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv('RRCA_baseflow.csv')

# Remove whitespace in col names
df.columns = df.columns.str.strip()

# create a modified date column to deal with idiosyncrasies
df['NewDate'] = df['Date'] - 693963

# create a positive Irrigation_pumping column to visualize the data better
df['PositiveIrrigation'] = df['Irrigation_pumping'] * -1

## Initial scatterplots to visualize the data

For each river segment, what is the relationship between observed flow and precipitation, irrigation, and evapotranspiration:

In [ ]:
# Get unique segment IDs
unique_segment_ids = df['Segment_id'].unique()

# Create a figure with subplots for each unique segment ID
fig, axs = plt.subplots(len(unique_segment_ids), 3, figsize=(15, 5 * len(unique_segment_ids)))

# Loop through each unique segment ID and plot the scatter plots
for i, segment_id in enumerate(unique_segment_ids):
    df_filtered = df[df['Segment_id'] == segment_id]
    # df_filtered.plot(kind='scatter', x='Irrigation_pumping', y='Observed', ax=axs[i, 0], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='PositiveIrrigation', y='Observed', ax=axs[i, 0], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='Precipitation', y='Observed', ax=axs[i, 1], title=f'Segment ID: {segment_id}')
    df_filtered.plot(kind='scatter', x='Evapotranspiration', y='Observed', ax=axs[i, 2], title=f'Segment ID: {segment_id}')

plt.tight_layout()
plt.show()

# Note: I tried flipping the axes here so that the zeros are on the x-axis, but it looked messier to me

How do these relationships change over time?

In [ ]:
# Get unique segment IDs
unique_segment_ids = df['Segment_id'].unique()

# For each segment, plot the observed baseflow over time
fig, axs = plt.subplots(len(unique_segment_ids), 1, figsize=(15, 5 * len(unique_segment_ids)))

# Loop through each unique segment ID and plot the scatter plots
for i, segment_id in enumerate(unique_segment_ids):
    df_filtered = df[df['Segment_id'] == segment_id]
    df_filtered.plot(kind='scatter', x='NewDate', y='Observed', ax=axs[i], title=f'Segment ID: {segment_id}')

plt.tight_layout()
plt.show()


### Is there a relationship between observed waterflow and evapotranspiration, precipitation, or irrigaion? 

Our tentative observations based on these scatterplots:
    * As there is more evapotranspiration in an area, the observed waterflow is lower
    * As more water is being pumped out for irrigation, the observed waterflow is lower
    * As there is more precipation in an area, ther _may_ be more observed waterflow

## Estimating ("Learning") Model Coefficients

In [ ]:
# create X and y
feature_cols = ['Irrigation_pumping', 'Precipitation', 'Evapotranspiration']
# feature_cols = ['Irrigation_pumping']
X = df[feature_cols]
y = df.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)

## Interpreting Model Coefficients

A "unit" increase in irrigation pumping is __associated with__ an 18.73476 "unit" increase in observed baseflow.

Wot?? That doesn't even make sense though because the charts seem to be saying that an increase in pumping _decreases_ the baseflow. Hmmm.

Ohhh, I see though. Irrigation right now is measured in negative values, meaning it's measuring the water being taken from the stream as a negative value. So as the amount of water being pumped _decreases_, the baseflow goes up. And that's with a "unit" increase of like 18.73476.

### How strong are the relationships between observed waterflow and evapotranspiration, precipitaiton, or irrigation?

## Using the Model for Prediction

(In other words: given irrigation, evapotranspiration, or precipitation, can baseflow be predicted?)

Use the Statsmodels to make the prediction:

In [ ]:
# you have to create a DataFrame since the Statsmodels formula interface expects it
# X_new = pd.DataFrame({'Irrigation_pumping': [50], 'Precipitation': [40], 'Evapotranspiration': [30]})
X_new = pd.DataFrame({'Irrigation_pumping': [50]})
display(X_new)

In [ ]:
# use the model to make predictions on a new value
lm.predict(X_new.values)

TODO: The above is kind of an arbitrary test of lm.predict right now, since it's using dummy values. I'm not sure what the usefulness of 50 irrigation, 40 precip, and 30 evapo would be.

## Plotting the Least Squares Line

Let's make predictions for the __smallest and largest observed values of x__, and then use the predicted values to plot the least squares line:

In [ ]:
# create a DataFrame with the minimum and maximum values of Irrigation_pumping, Precipitation, and Evapotranspiration
# X_new = pd.DataFrame({'Irrigation_pumping': [df.Irrigation_pumping.min(), df.Irrigation_pumping.max()], 'Precipitation': [df.Precipitation.min(), df.Precipitation.max()], 'Evapotranspiration': [df.Evapotranspiration.min(), df.Evapotranspiration.max()]})
X_new = pd.DataFrame({'Irrigation_pumping': [df.Irrigation_pumping.min(), df.Irrigation_pumping.max()]})
X_new

In [ ]:
# make predictions for those x values and store them
preds = lm.predict(X_new)
preds

In [ ]:
# use seaborn to plot the observed data
import seaborn as sns

sns.regplot(x='Irrigation_pumping', y='Observed', data=df, line_kws={'color': 'red'})

## Confidence in our Model

Use __statsmodels__ to compute the __confidence interval__.

In [ ]:
# this is the standard import if you're using "formula notation"
import statsmodels.formula.api as smf

# create a fitted model in one line
lm = smf.ols(formula='Observed ~ Irrigation_pumping', data=df).fit()

# print the coefficients
display(lm.params)

# print the confidence intervals for the model coefficients
display(lm.conf_int())

This says that when `Irrigaion_pumping` is zero, the expected value of Observed is 25.007945.

For each additional unit of `Irrigation_pumping`, Observed increases by 11.257441. (Since the amount of water being pumped out for irrigation decreses as `Irrigation_pumping` increases, this is saying that one unit _less_ of water being pumped out of the river for irrigation results in 11.257 units more of Observed.)

We can plot the confidence intervals using Seaborn. The confidence intervals are computed a little differently: the data is binned according to the independent variable and then a band containing 95% of the means of the bin is shown.

In [ ]:
import seaborn as sns

# take a sample of the dataset to make the plot more manageable
df_sample = df.sample(n=1000)

# Plot the sampled data with the confidence interval
plt.figure()
ax = sns.regplot(x='Irrigation_pumping', y='Observed', data=df_sample, ci=95, scatter_kws={'s': 1})

plt.figure()
ax = sns.regplot(x='Evapotranspiration', y='Observed', data=df_sample, ci=95, scatter_kws={'s': 1})

plt.figure()
ax = sns.regplot(x='Precipitation', y='Observed', data=df_sample, ci=95, scatter_kws={'s': 1})

The shaded region shows all possible regression curves within the 95% confidence interval.

## Hypothesis Testing and p-values

As it relates to model coefficients, here is the conventional hypothesis test:
- **null hypothesis:** There is no relationship between irrigation pumping and observed flow (and thus $\beta_1$ equals zero)
- **alternative hypothesis:** There is a relationship between irrigation pumping and observed flow (and thus $\beta_1$ is not equal to zero)

We reject the null (and thus believe the alternative) if the 95% confidence interval __does not include zero__. The p-value represents the probability that the coefficient is actually zero:

In [ ]:
# lm = smf.ols(formula='Observed ~ Irrigation_pumping + Precipitation + Evapotranspiration', data=df).fit()
lm = smf.ols(formula='Observed ~ Irrigation_pumping', data=df).fit()
# print the p-values for the model coefficients
display(lm.pvalues)

lm = smf.ols(formula='Observed ~ Evapotranspiration', data=df).fit()
display(lm.pvalues)

lm = smf.ols(formula='Observed ~ Precipitation', data=df).fit()
display(lm.pvalues)

If the 95% confidence interval **includes zero**, the p-value for that coefficient will be **greater than 0.05**. If the 95% confidence interval **does not include zero**, the p-value will be **less than 0.05**. Thus, a p-value less than 0.05 is one way to decide whether there is likely a relationship between the feature and the response. (Again, using 0.05 as the cutoff is just a convention.)

In this case, the p-value for `Irrigation_pumping` is far less than 0.05, and so we **believe** that there is a relationship between irrigation pumping and observed flow.

## How Well Does the Model Fit the data? (R-squared)

The most common way to evaluate the overall fit of a linear model is by the **R-squared** value. R-squared is the **proportion of variance explained**, meaning the proportion of variance in the observed data that is explained by the model, or the reduction in error over the **null model**. (The null model just predicts the mean of the observed response, and thus it has an intercept and no slope.)

$R^2 = 1 - \frac{\text{residual}}{\text{total variation}} = \frac{\sum_{i=1}^N (y_i-\hat{y}_i)^2}{\sum_{i=1}^N (y_i-\overline{y})^2}$

where $\overline{y}$ is the average value and $\hat{y}_i$ is the predicted value.

R-squared is between 0 and 1, and higher is better because it means that more variance is explained by the model. This is a way of normalizing the raw residual.

Let's calculate the R-squared value for our simple linear model:

In [ ]:
# Print the R-squared value for the model using statsmodels
lm = smf.ols(formula='Observed ~ Irrigation_pumping', data=df).fit()
display(lm.rsquared)

# Print the R-squared value for the model using sklearn
lm = LinearRegression()
lm.fit(df[['Irrigation_pumping']], df.Observed)
display(lm.score(df[['Irrigation_pumping']], df.Observed))

Is that a "good" R-squared value? It's hard to say. The threshold for a good R-squared value depends widely on the domain. Therefore, it's most useful as a tool for comparing different models.

## Multiple Linear Regression

Simple linear regression can easily be extended to include multiple features. This is called __multiple linear regression__.

Each $x$ represents a different feature, and each feature has its own coefficient.

Use sklearn to estimate these coefficients:

In [ ]:
# create X and y
feature_cols = ['Irrigation_pumping', 'Precipitation', 'Evapotranspiration']
X = df[feature_cols]
y = df.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
display(list(zip(feature_cols, lm.coef_)))

How do we interpret these results? For a given amount of irrigation pumping, precipitation, and evapotranspiration, an __increase of 1 unit of irrigation pumping__ is associated with an __increase in observed waterflow of 18.73476__.

A lot of the information we have been reviewing piece-by-piece is available in the model summary output using statsmodels:

In [ ]:
# create a fitted model with all three features
lm = smf.ols(formula='Observed ~ Irrigation_pumping + Precipitation + Evapotranspiration', data=df).fit()

# print the coefficients
display(lm.params)

# print the summary of the fitted model
display(lm.summary())

What are a few things we learn from this output?
* ...
* ...
* ...

## Feature Selection

Which features are we going to include in our linear model?

According to the scatterplots we did of each segment of the river, irrigation pumping and evapostranspiration seem to be correlated with observed flow. Precipation doesn't have any apparent correlation with observed flow.